<a href="https://colab.research.google.com/github/yuvasri-185/emotion-detection/blob/main/Emotion_Detection_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio scikit-learn nltk pandas numpy matplotlib seaborn


In [ ]:
import pandas as pd
import numpy as np
import nltk
import re
import warnings

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import gradio as gr

warnings.filterwarnings("ignore")

nltk.download("punkt")
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

print("✅ Setup complete")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ Setup complete


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"[^a-z\s]", "", text)

    tokens = text.split()  # NO NLTK tokenizer
    stop_words = set(stopwords.words("english"))
    tokens = [w for w in tokens if w not in stop_words and len(w) > 2]

    return " ".join(tokens)


In [ ]:
def create_dataset():
    emotions = {
        "happy": ["I am happy","Feeling joyful","So excited","Best day ever"] * 1000,
        "sad": ["I feel sad","Very lonely","Feeling depressed","Feeling low"] * 100,
        "angry": ["I am angry","So frustrated","This makes me mad","Very annoyed"] * 100,
        "fear": ["I am scared","Feeling anxious","Very afraid","Feeling nervous"] * 100,
        "surprise": ["Wow amazing","So surprising","I didn't expect this","Shocking news"] * 100,
        "neutral": ["The meeting is today","I am working","Task completed","Normal day"] * 100,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 50,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "sad": ["I feel sad","Feeling down","Very unhappy","Crying"] * 100,
        "sad": ["I feel sad","Feeling down","Very unhappy","Crying"] * 100,
        "angry": ["I am angry","So mad","This is annoying","I hate this"] * 25,
        "fear": ["I am scared","Feeling anxious","I am afraid","So nervous"] * 25,
        "surprise": ["Wow unbelievable","That was shocking","Unexpected","So surprised"] * 25,
        "fear": ["I am scared","Feeling anxious","I am afraid","So nervous"] * 25,
        "surprise": ["Wow unbelievable","That was shocking","Unexpected","So surprised"] * 25,
        "neutral": ["The meeting is today","I am working","Task completed","Normal day"] * 100,
        "neutral": ["The meeting is today","I am working","Task completed","Normal day"] * 100,
        "neutral": ["The meeting is today","I am working","Task completed","Normal day"] * 100,
        "happy": ["I am happy","Feeling joyful","So excited","Best day ever"] * 1000,
        "sad": ["I feel sad","Very lonely","Feeling depressed","Feeling low"] * 100,
        "angry": ["I am angry","So frustrated","This makes me mad","Very annoyed"] * 100,
        "fear": ["I am scared","Feeling anxious","Very afraid","Feeling nervous"] * 100,
        "surprise": ["Wow amazing","So surprising","I didn't expect this","Shocking news"] * 100,
        "neutral": ["The meeting is today","I am working","Task completed","Normal day"] * 100,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 250,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 500,
        "happy": ["I am happy","Feeling joyful","So excited","Best day"] * 25,
        "sad": ["I feel sad","Feeling down","Very unhappy","Crying"] * 100,
        "sad": ["I feel sad","Feeling down","Very unhappy","Crying"] * 100,
        "angry": ["I am angry","So mad","This is annoying","I hate this"] * 25,
        "fear": ["I am scared","Feeling anxious","I am afraid","So nervous"] * 25,
        "surprise": ["Wow unbelievable","That was shocking","Unexpected","So surprised"] * 25,
        "fear": ["I am scared","Feeling anxious","I am afraid","So nervous"] * 25,
        "surprise": ["Wow unbelievable","That was shocking","Unexpected","So surprised"] * 250


    }

    text, emotion = [], []
    for emo, sentences in emotions.items():
        for s in sentences:
            text.append(s)
            emotion.append(emo)

    return pd.DataFrame({"text": text, "emotion": emotion})

df = create_dataset()
df["clean_text"] = df["text"].apply(preprocess_text)

df.head()


,text,emotion,clean_text
0,I am happy,happy,happy
1,Feeling joyful,happy,feeling joyful
2,So excited,happy,excited
3,Best day,happy,best day
4,I am happy,happy,happy


In [ ]:
df["emotion"].value_counts()


,count
emotion,
surprise,1000
sad,400
neutral,400
happy,100
fear,100
angry,100


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"],
    df["emotion"],
    test_size=0.2,
    random_state=42,
    stratify=df["emotion"]
)


In [ ]:
vectorizer = TfidfVectorizer(
    ngram_range=(1,2),
    max_features=8000
)

X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [ ]:
model = LogisticRegression(
    max_iter=2000,
    class_weight="balanced"
)

model.fit(X_train_vec, y_train)
print("✅ Model trained")


✅ Model trained


In [ ]:
y_pred = model.predict(X_test_vec)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00        20
        fear       1.00      1.00      1.00        20
       happy       1.00      1.00      1.00        20
     neutral       1.00      1.00      1.00        40
         sad       1.00      1.00      1.00        80
    surprise       1.00      1.00      1.00        20

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



In [ ]:
def predict_emotion(text):
    clean = preprocess_text(text)
    vec = vectorizer.transform([clean])
    probs = model.predict_proba(vec)[0]
    emotion = model.classes_[np.argmax(probs)]

    return emotion, max(probs)


In [ ]:
def get_emoji(emotion):
    emojis = {
        "happy": "😊",
        "sad": "😢",
        "angry": "😠",
        "fear": "😨",
        "surprise": "😲",
        "neutral": "😐"
    }
    return emojis[emotion]


In [ ]:
def analyze_emotion(text):
    if not text.strip():
        return "⚠️ Please enter text"

    emotion, confidence = predict_emotion(text)
    emoji = get_emoji(emotion)

    return f"{emoji} Emotion: {emotion.upper()}\n📊 Confidence: {confidence:.2%}"


In [ ]:
demo = gr.Interface(
    fn=analyze_emotion,
    inputs=gr.Textbox(lines=4, placeholder="Enter your message"),
    outputs=gr.Textbox(lines=3),
    title="🎭 Emotion Detection System",
    description="Detect emotion from text using Machine Learning"
)


In [ ]:
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8131e07669810ae396.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
